In [1]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
import json

import os
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-18.0.2'
os.environ['SPARK_HOME'] = 'C:\Program Files\spark-3.3.0-bin-hadoop3'
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [2]:
#Create a Python program to GET (consume) data from the above API endpoint for the loan application dataset.
got = requests.get('https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json')

#Create a spark session

spark = SparkSession.builder.appName('Loan App').getOrCreate()

json_data = json.loads(got.content)
json_data[0]
df=pd.read_json(got.content)
df = spark.createDataFrame(df)
#df2 = pd.read_json(got.content)
df.createOrReplaceTempView("df")

#df.show()
#Read JSON and assign to pyspark dataframes
#rdd = spark.sparkContext.parallelize([got.content])
#df = spark.read.json(json_data)
df.printSchema()



root
 |-- Application_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Application_Status: string (nullable = true)



In [3]:
#Find the status code of the above API endpoint.
print(got.status_code)

200


In [4]:
df.show()

+--------------+------+-------+----------+------------+-------------+--------------+-------------+------+------------------+
|Application_ID|Gender|Married|Dependents|   Education|Self_Employed|Credit_History|Property_Area|Income|Application_Status|
+--------------+------+-------+----------+------------+-------------+--------------+-------------+------+------------------+
|      LP001002|  Male|     No|         0|    Graduate|           No|             1|        Urban|medium|                 Y|
|      LP001003|  Male|    Yes|         1|    Graduate|           No|             1|        Rural|medium|                 N|
|      LP001005|  Male|    Yes|         0|    Graduate|          Yes|             1|        Urban|   low|                 Y|
|      LP001006|  Male|    Yes|         0|Not Graduate|           No|             1|        Urban|   low|                 Y|
|      LP001008|  Male|     No|         0|    Graduate|           No|             1|        Urban|medium|                 Y|


In [5]:
df.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_loan_application") \
  .option("driver", "com.mysql.jdbc.Driver") \
  .option("user", "root") \
  .option("password", "db") \
  .save()

In [6]:

df1 = df.toPandas()
import plotly.express as px



In [28]:
#Find and plot the percentage of applications approved for self-employed applicants.
approved = df1[df1['Application_Status']=='Y']
percent_accepted = len(approved) / len(df1)
percent_accepted

fig = px.pie(values=[percent_accepted, 1.0], names=['Approved', 'Total'])
fig.show()

In [58]:
#Find the percentage of rejection for married male applicants.
import numpy as np
male= np.logical_and(df1['Gender']=='Male', df1['Application_Status']=='Y' )
#& df1[df1['Application_Status']=='Y']
np.count_nonzero(male)
len(male)

fig = px.bar(male)
fig.update_layout(barmode='stack', )
fig.show()

 #& [df1['Application_Status']=='Y']





In [61]:
#Reading data from database because we need the credit table

url = "jdbc:mysql://localhost:3306/creditcard_capstone"
driver = "com.mysql.jdbc.Driver"
user = "root"
password = "db"


credit_df =  spark.read\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", url)\
    .option("user", user)\
    .option("password", password)\
    .option("dbtable", "CDW_SAPP_CREDIT")\
    .load()

credit_df = credit_df.toPandas()

#grouped = credit_df.groupby('BRANCH_CODE')['']


In [103]:
#Find and plot the top three months with the largest transaction data.
dates = credit_df[['TIMEID', 'TRANSACTION_VALUE']]
dates['TIMEID'] = pd.to_datetime(dates['TIMEID'], format='%Y%m%d').dt.month
dates


tgroup = dates.groupby('TIMEID')['TRANSACTION_VALUE'].sum().reset_index()
tgroup

fig = px.line(tgroup, x='TIMEID', y='TRANSACTION_VALUE', markers=True)
fig.show()

C:\Users\kayys\AppData\Local\Temp\ipykernel_24560\2861480095.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [84]:
#Find and plot which branch processed the highest total dollar value of healthcare transactions.

a = np.where(credit_df['TRANSACTION_TYPE']=='Healthcare')
health = credit_df.iloc[a]
grouped = health.groupby('BRANCH_CODE')['TRANSACTION_VALUE'].sum().reset_index()

grouped = grouped.sort_values(by='TRANSACTION_VALUE', ascending=False)
grouped
fig = px.scatter(grouped, x='BRANCH_CODE', y='TRANSACTION_VALUE', color='TRANSACTION_VALUE', size='TRANSACTION_VALUE')
fig.show()